### Problem 1 (50 points) 

Vapor-liquid equilibria data are correlated using two adjustable parameters $A_{12}$ and $A_{21}$ per binary
mixture. For low pressures, the equilibrium relation can be formulated as:

$$
\begin{aligned}
p = & x_1\exp\left(A_{12}\left(\frac{A_{21}x_2}{A_{12}x_1+A_{21}x_2}\right)^2\right)p_{water}^{sat}\\
& + x_2\exp\left(A_{21}\left(\frac{A_{12}x_1}{A_{12}x_1+A_{21}x_2}\right)^2\right)p_{1,4 dioxane}^{sat}.
\end{aligned}
$$

Here the saturation pressures are given by the Antoine equation

$$
\log_{10}(p^{sat}) = a_1 - \frac{a_2}{T + a_3},
$$

where $T = 20$($^{\circ}{\rm C}$) and $a_{1,2,3}$ for a water - 1,4 dioxane
system is given below.

|             | $a_1$     | $a_2$      | $a_3$     |
|:------------|:--------|:---------|:--------|
| Water       | 8.07131 | 1730.63  | 233.426 |
| 1,4 dioxane | 7.43155 | 1554.679 | 240.337 |


The following table lists the measured data. Recall that in a binary system $x_1 + x_2 = 1$.

|$x_1$ | 0.0 | 0.1 | 0.2 | 0.3 | 0.4 | 0.5 | 0.6 | 0.7 | 0.8 | 0.9 | 1.0 |
|:-----|:--------|:---------|:--------|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|
|$p$| 28.1 | 34.4 | 36.7 | 36.9 | 36.8 | 36.7 | 36.5 | 35.4 | 32.9 | 27.7 | 17.5 |

Estimate $A_{12}$ and $A_{21}$ using data from the above table: 

1. Formulate the least square problem; 
2. Since the model is nonlinear, the problem does not have an analytical solution. Therefore, solve it using the gradient descent or Newton's method implemented in HW1; 
3. Compare your optimized model with the data. Does your model fit well with the data?




#### Solution 

1.
The given problem can be formulated as a Least Squares problem as follows:

$$
\begin{aligned}
 \underset{A_{12},A_{21}}{\text{minimize: }} \sum_{i=1}^{N=11}(p(x_i;A_{12},A_{21})-p_i)^2
\end{aligned}
$$

<br>
2.

Saturation pressure for water:

$$
\log_{10}(p_{water}^{sat}) = a_1 - \frac{a_2}{T + a_3} = 8.07131 - \frac{1730.63}{20 + 233.426} = 1.24237
$$

<br>
$$
\Rightarrow p_{water}^{sat} = 17.47325
$$


Saturation pressure for 1,4dioxane:

$$
\log_{10}(p_{1,4dioxane}^{sat}) = a_1 - \frac{a_2}{T + a_3} = 7.43155 - \frac{1554.679}{20 + 240.337} = 1.45975
$$

<br> 	
$$
\Rightarrow p_{water}^{sat} = 28.82409
$$

<br>
With $x_2=1-x_1$, the equilibrium relation becomes:

$$
\begin{aligned}
p = & x_1\exp\left(A_{12}\left(\frac{A_{21}(1-x_1)}{A_{12}x_1+A_{21}(1-x_1)}\right)^2\right)p_{water}^{sat}\\
& + (1-x_1)\exp\left(A_{21}\left(\frac{A_{12}x_1}{A_{12}x_1+A_{21}(1-x_1)}\right)^2\right)p_{1,4 dioxane}^{sat}.
\end{aligned}
$$


In [35]:
import numpy as np
import torch 



def saturation_pressure(T, a1, a2, a3):
    """Compute the saturation pressure."""

    log_p = a1 - (a2/(T + a3))
    p = 10**log_p
    return p

def gradient_descent(x1, p, a, eps, max_iter):
    """ """

    # Randomly initialize weights
    A12 = torch.randn((), requires_grad=True, device=device, dtype=torch.float) 
    A21 = torch.randn((), requires_grad=True, device=device, dtype=torch.float) 


    for i in range(1000):

        # Compute p prediction (Forward pass)
        p_pred = x1*torch.exp(A12*(A21*(1-x1)/(A12*x1+A21*(1-x1)))**2)*water_sat_p + \
                (1-x1)*torch.exp(A21*(A12*x1/(A12*x1+A21*(1-x1)))**2)*dioxane_sat_p
        
        # Compute loss
        loss = (p_pred - p).pow(2).sum()

        # Print loss every 100 iterations
        if i % 100 == 0:
            print(i, loss.item())
        
        # Compute gradient of the loss wrt all the learnable parameters of the model
        loss.backward()

        # Update the weights using Gradient Descent
        with torch.no_grad():  # We don't need the GD algorithm itself to be differentiable wrt x1
            
            A12 -= a * A12.grad
            A21 -= a * A21.grad
        
            # Clear the gradients (so they don't accumulate)
            A12.grad.zero_()
            A21.grad.zero_()
    
    return (A12.item(), A21.item())





if __name__ == '__main__':

    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # Parameters
    # --------------------------------------------------------------------------
    eps = 1e-3                       # Tolerance (termination criterion)
    max_iter = 15000                 # Maximum iterations before halting

    # Compute saturation pressures
    # --------------------------------------------------------------------------
    water_sat_p = saturation_pressure(T=20, a1=8.07131, a2=1730.63, a3=233.426)
    dioxane_sat_p = saturation_pressure(T=20, a1=7.43155, a2=1554.679, a3=240.337)


    
    # Input data
    x1 = torch.tensor([0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], device=device, dtype=torch.float)
    # Output data
    p = torch.tensor([28.1, 34.4, 36.7, 36.9, 36.8, 36.7, 36.5, 35.4, 32.9, 27.7, 17.5], device=device, dtype=torch.float)

    sol = gradient_descent(x1, p, 0.01, eps, max_iter)

    print(sol)


    # print('\nNumber of iterations: {}'.format(i))
    # print('The solution is: {}'.format(x.data.numpy()))
    # print('Final loss value: {}'.format(loss.data.numpy()))

    




0 2150.4189453125
100 11028.6259765625
200 11028.6259765625
300 11028.6259765625
400 11028.6259765625
500 11028.6259765625
600 11028.6259765625
700 11028.6259765625
800 11028.6259765625
900 11028.6259765625
(-11515368308736.0, -1405378232320.0)


---
### Problem 2 (50 points) 

Solve the following problem using Bayesian Optimization:
$$
    \min_{x_1, x_2} \quad \left(4-2.1x_1^2 + \frac{x_1^4}{3}\right)x_1^2 + x_1x_2 + \left(-4 + 4x_2^2\right)x_2^2,
$$
for $x_1 \in [-3,3]$ and $x_2 \in [-2,2]$. A tutorial on Bayesian Optimization can be found [here](https://thuijskens.github.io/2016/12/29/bayesian-optimisation/).



#### Solution 

